In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
from os import listdir
from os.path import isfile, join
from downloaded_data_preprocessing import info_extraction

### READ ALL FILE

In [3]:
mypath = '../scraping/data_download'

In [4]:
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

### EXTRA INFO 

In [5]:
import json 

img_list = []
f = open("../scraping/status_img_download/img_src.txt", "r")
while f:
    line  = f.readline()
    if line!= ' ' and line!='':
        img_list.append(json.loads(line.replace('\n','')))
    if line == "":
            break
f.close() 

In [6]:
extra_data = pd.DataFrame(img_list)

In [7]:
extra_data.head()

,Stagione,Nome,Id,Ruolo,Squadra,Src
0,2021_22,Adamonis Marius,5502,P,Lazio,https://content.fantacalcio.it/web/campioncini...
1,2021_22,Aresti Simone,1946,P,Cagliari,https://content.fantacalcio.it/web/campioncini...
2,2021_22,Audero Emil,761,P,Sampdoria,https://content.fantacalcio.it/web/campioncini...
3,2021_22,Bardi Francesco,799,P,Bologna,https://content.fantacalcio.it/web/campioncini...
4,2021_22,Belec Vid,720,P,Salernitana,https://content.fantacalcio.it/web/campioncini...


### CREATE DATAFRAME FOR EACH PLAYER

In [8]:
dfs_info = []
dfs_data = []

In [9]:
len(onlyfiles)

499

In [10]:
for file in tqdm(onlyfiles):
    
    stagione = file.split('^')[1].split(' ')[1].split(')')[0]
    nome = file.replace('^(Stagione '+stagione+').xlsx','')
    data = pd.read_excel(mypath + '/' +file, skiprows=[0,1]).rename(columns={'Unnamed: 1':'Partita'})
    status = pd.read_csv('../scraping/status_img_download/'+nome+'^(Status ' + stagione +').csv',index_col=[0])
    
    data['Ruolo'] = extra_data.loc[extra_data.Nome == nome,'Ruolo'].values[0]
    data['Squadra'] = extra_data.loc[extra_data.Nome == nome,'Squadra'].values[0]
    data['Id'] = extra_data.loc[extra_data.Nome == nome,'Id'].values[0]
    data = data.merge(status, left_on='Giornata', right_on='Giornata')

    info,table = info_extraction(data,nome,stagione)
    
    img = extra_data.loc[extra_data.Nome == nome,'Src'].values[0]
    info['img'] = img
    
    dfs_info.append(info)
    dfs_data.append(table)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [00:27<00:00, 18.22it/s]


In [11]:
player_info = pd.DataFrame(pd.concat(dfs_info))
player_info.index = range(0,len(player_info))

In [12]:
player_history = pd.DataFrame(pd.concat(dfs_data))
player_history.index = range(0,len(player_history))

In [13]:
player_history.fillna(0).to_csv('../dataset/player_history^(Stagione '+stagione+').csv')

### ADD INFO OF BAD URLS PLAYER

In [14]:
import json 

error_scraping_player = []
f = open("../scraping/status_img_download/error_scraping_player.txt", "r")
while f:
    line  = f.readline()
    if line!= ' ' and line!='':
        error_scraping_player.append(json.loads(line.replace('\n','')))
    if line == "":
            break
f.close() 

In [15]:
error_scraping_player_df = pd.DataFrame(error_scraping_player).drop_duplicates()

In [16]:
error_scraping_player_df = error_scraping_player_df.rename(columns={'Src':'img'})

In [17]:
error_scraping_player_df['Gol subiti'] = error_scraping_player_df['Gol']
error_scraping_player_df.loc[error_scraping_player_df['Ruolo'] == 'P', 'Gol' ] = 0
error_scraping_player_df.loc[error_scraping_player_df['Ruolo'] != 'P', 'Gol subiti' ] = 0

In [18]:
for c in player_info.columns.tolist():
    if c not in error_scraping_player_df.columns:
        error_scraping_player_df[c] = None

In [19]:
player_info = pd.concat([player_info, error_scraping_player_df])
player_info = player_info.set_index('Id')

In [22]:
player_info.fillna(0).to_csv('../dataset/player_info^(Stagione '+stagione+').csv')